# Keyword querying and filtering

This interactive notebook will introduce you to the basic **Elasticsearch Queries**, using the official `Elastic.Clients.Elasticsearch` .NET  client. Before getting started on this section you should work through our quick start, as you will be using the same dataset.

## Initialize the Elasticsearch client

Now, we need to initialize the Elasticsearch client. We will use the [Elasticsearch client for .NET](https://www.elastic.co/guide/en/elasticsearch/client/net-api/current/index.html) to connect to Elasticsearch.


#### Install packages and import modules

In [ ]:
#r "nuget: Elastic.Clients.Elasticsearch"

#!import ./Utils.cs
#!import ../_infra/get-connection-string.ipynb

#### Initialize Client


In [ ]:
using Elastic.Transport;
using Elastic.Clients.Elasticsearch;
using Elastic.Transport.Products.Elasticsearch;

var elasticSettings = new ElasticsearchClientSettings(connectionString)
    .DisableDirectStreaming()
    .ServerCertificateValidationCallback(CertificateValidations.AllowAll);

var client = new ElasticsearchClient(elasticSettings);

#### Define Model

In [ ]:
using System.Text.Json.Serialization;

public class Book
{
    public string Title { get; set; }

    public string Summary { get; set; }

    public List<string> Authors { get; set; }

    public DateTime publish_date { get; set; }

    public int num_reviews { get; set; }

    public string Publisher { get; set; }
}

void PrettyPrint(SearchResponse<Book> searchResponse) => searchResponse.Hits
    .Select(x => new { 
        Title = x.Source.Title,
        Score = x.Score,
        Summary = x.Source.Summary,
        Id = x.Id,
        Publisher = x.Source.Publisher,
        Authors = x.Source.Authors,
        Reviews = x.Source.num_reviews,
    })
    .DisplayTable();

## Querying

🔐 NOTE: to run the queries that follow you need the book_index dataset from our [00-quick-start.ipynb](./00-quick-start.ipynb). If you haven't worked through the quick start, please follow the steps described there to create an Elasticsearch deployment with the dataset in it, and then come back to run the queries here.

In the query context, a query clause answers the question *“How well does this document match this query clause?”*. In addition to deciding whether or not the document matches, the query clause also calculates a relevance score in the `_score` metadata field.

### Full text queries
Full text queries enable you to search analyzed text fields such as the body of an email. The query string is processed using the same analyzer that was applied to the field during indexing.

* **match**. The standard query for performing full text queries, including fuzzy matching and phrase or proximity queries.
* **multi-match**. The multi-field version of the match query.

#### Match query
Returns documents that `match` a provided text, number, date or boolean value. The provided text is analyzed before matching.

The `match` query is the standard query for performing a full-text search, including options for fuzzy matching. 

> 💡 [Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-query-ex-request)

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Match(m => m
            .Field(f => f.Summary)
            .Query("guide")
        )
    )
    .Size(5)
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
The Pragmatic Programmer: Your Journey to Mastery,0.7042277,A guide to pragmatic programming for software engineers and developers,DiYfqZIBTvBCTLU88r8R,addison-wesley,"[ andrew hunt, david thomas ]",30
Python Crash Course,0.7042277,"A fast-paced, no-nonsense guide to programming in Python",DyYfqZIBTvBCTLU88r8R,no starch press,[ eric matthes ],42
The Clean Coder: A Code of Conduct for Professional Programmers,0.6771651,A guide to professional conduct in the field of software engineering,FSYfqZIBTvBCTLU88r8R,prentice hall,[ robert c. martin ],20
Clean Code: A Handbook of Agile Software Craftsmanship,0.62883455,"A guide to writing code that is easy to read, understand and maintain",ESYfqZIBTvBCTLU88r8R,prentice hall,[ robert c. martin ],55
Design Patterns: Elements of Reusable Object-Oriented Software,0.62883455,Guide to design patterns that can be used in any object-oriented language,FCYfqZIBTvBCTLU88r8R,addison-wesley,"[ erich gamma, richard helm, ralph johnson, john vlissides ]",45


#### Multi-match query

The `multi_match` query builds on the match query to allow multi-field queries.

> 💡 [Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html).

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .MultiMatch(m => m
            .Fields(Fields.FromStrings(["summary", "title"]))
            .Query("javascript")
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Eloquent JavaScript,2.0307527,A modern introduction to programming,EyYfqZIBTvBCTLU88r8R,no starch press,[ marijn haverbeke ],38
JavaScript: The Good Parts,1.7064086,A deep dive into the parts of JavaScript that are essential to writing maintainable code,FiYfqZIBTvBCTLU88r8R,oreilly,[ douglas crockford ],51
You Don't Know JS: Up & Going,1.6360576,Introduction to JavaScript and programming as a whole,EiYfqZIBTvBCTLU88r8R,oreilly,[ kyle simpson ],36


➕ Individual fields can be boosted with the **caret (^) notation**. Note in the following query how the score of the results that have "JavaScript" in their title is multiplied.

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .MultiMatch(m => m
            .Fields(Fields.FromStrings(["summary", "title^3"]))
            .Query("javascript")
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Eloquent JavaScript,6.0922585,A modern introduction to programming,EyYfqZIBTvBCTLU88r8R,no starch press,[ marijn haverbeke ],38
JavaScript: The Good Parts,5.1192265,A deep dive into the parts of JavaScript that are essential to writing maintainable code,FiYfqZIBTvBCTLU88r8R,oreilly,[ douglas crockford ],51
You Don't Know JS: Up & Going,1.6360576,Introduction to JavaScript and programming as a whole,EiYfqZIBTvBCTLU88r8R,oreilly,[ kyle simpson ],36


## Term-level Queries

You can use term-level queries to find documents based on precise values in structured data. Examples of structured data include date ranges, IP addresses, prices, or product IDs.

### Term search
Returns document that contain exactly the search term.

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Term(t => t
            .Field(f => f.Publisher)
            .Value("addison-wesley")
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
The Pragmatic Programmer: Your Journey to Mastery,1.4816045,A guide to pragmatic programming for software engineers and developers,DiYfqZIBTvBCTLU88r8R,addison-wesley,"[ andrew hunt, david thomas ]",30
Design Patterns: Elements of Reusable Object-Oriented Software,1.4816045,Guide to design patterns that can be used in any object-oriented language,FCYfqZIBTvBCTLU88r8R,addison-wesley,"[ erich gamma, richard helm, ralph johnson, john vlissides ]",45


#### Range search

Returns documents that contain terms within a provided range.

The following example returns books that have at least 45 reviews.

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Range(r => r
            .NumberRange(nr => nr.Field(f => f.num_reviews).Gte(45))
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Clean Code: A Handbook of Agile Software Craftsmanship,1,"A guide to writing code that is easy to read, understand and maintain",ESYfqZIBTvBCTLU88r8R,prentice hall,[ robert c. martin ],55
Design Patterns: Elements of Reusable Object-Oriented Software,1,Guide to design patterns that can be used in any object-oriented language,FCYfqZIBTvBCTLU88r8R,addison-wesley,"[ erich gamma, richard helm, ralph johnson, john vlissides ]",45
JavaScript: The Good Parts,1,A deep dive into the parts of JavaScript that are essential to writing maintainable code,FiYfqZIBTvBCTLU88r8R,oreilly,[ douglas crockford ],51


#### Prefix search

Returns documents that contain a specific prefix in a provided field.

> 💡 [Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-prefix-query.html).

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Prefix(p => p
            .Field(f => f.Title)
            .Value("java")
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Eloquent JavaScript,1,A modern introduction to programming,EyYfqZIBTvBCTLU88r8R,no starch press,[ marijn haverbeke ],38
JavaScript: The Good Parts,1,A deep dive into the parts of JavaScript that are essential to writing maintainable code,FiYfqZIBTvBCTLU88r8R,oreilly,[ douglas crockford ],51


#### Fuzzy search

Returns documents that contain terms similar to the search term, as measured by a *Levenshtein edit* distance.

An edit distance is the number of one-character changes needed to turn one term into another. These changes can include:

* Changing a character (box → fox)
* Removing a character (black → lack)
* Inserting a character (sic → sick)
* Transposing two adjacent characters (act → cat)

> 💡 [Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-fuzzy-query.html).

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Fuzzy(f => f
            .Field(ff => ff.Title)
            .Value("pyvascript")
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Eloquent JavaScript,1.6246022,A modern introduction to programming,EyYfqZIBTvBCTLU88r8R,no starch press,[ marijn haverbeke ],38
JavaScript: The Good Parts,1.3651271,A deep dive into the parts of JavaScript that are essential to writing maintainable code,FiYfqZIBTvBCTLU88r8R,oreilly,[ douglas crockford ],51


### Combining Query Conditions

Compound queries wrap other compound or leaf queries, either to combine their results and scores, or to change their behaviour. They also allow you to switch from query to filter context, but that will be covered later in the Filtering section.

#### bool.must (AND)
The clauses must appear in matching documents and will contribute to the score. This effectively performs an "AND" logical operation on the given sub-queries.

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Bool(b => b
            .Must(m => m
                .Term(t => t
                    .Field(f => f.Publisher)
                    .Value("addison-wesley")
                ),
                m => m
                .Term(t => t
                    .Field(f => f.Authors)
                    .Value("richard helm")
                )
            )
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Design Patterns: Elements of Reusable Object-Oriented Software,3.788629,Guide to design patterns that can be used in any object-oriented language,FCYfqZIBTvBCTLU88r8R,addison-wesley,"[ erich gamma, richard helm, ralph johnson, john vlissides ]",45


#### bool.should (OR)

The clause should appear in the matching document. This performs an "OR" logical operation on the given sub-queries.

In [ ]:
// shoud

var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Bool(b => b
            .Should(m => m
                .Term(t => t
                    .Field(f => f.Publisher)
                    .Value("addison-wesley")
                ),
                m => m
                .Term(t => t
                    .Field(f => f.Authors)
                    .Value("richard helm")
                )
            )
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Design Patterns: Elements of Reusable Object-Oriented Software,3.788629,Guide to design patterns that can be used in any object-oriented language,FCYfqZIBTvBCTLU88r8R,addison-wesley,"[ erich gamma, richard helm, ralph johnson, john vlissides ]",45
The Pragmatic Programmer: Your Journey to Mastery,1.4816045,A guide to pragmatic programming for software engineers and developers,DiYfqZIBTvBCTLU88r8R,addison-wesley,"[ andrew hunt, david thomas ]",30


## Filtering
In a filter context, a query clause answers the question “Does this document match this query clause?” The answer is a simple Yes or No — no scores are calculated. Filter context is mostly used for filtering structured data, for example:

* Does this `timestamp` fall into the range 2015 to 2016?
* Is the `status` field set to *"published"*?

Filter context is in effect whenever a query clause is passed to a `filter` parameter, such as the `filter` or `must_not` parameters in the `bool` query.

> 💡 [Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html).

### bool.filter
The clause (query) must appear for the document to be included in the results. Unlike query context searches such as term, bool.must or bool.should, a matching score isn't calculated because filter clauses are executed in filter context.

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Bool(b => b
            .Filter(m => m
                .Term(t => t
                    .Field(f => f.Publisher)
                    .Value("prentice hall")
                )
            )
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Clean Code: A Handbook of Agile Software Craftsmanship,0,"A guide to writing code that is easy to read, understand and maintain",ESYfqZIBTvBCTLU88r8R,prentice hall,[ robert c. martin ],55
The Clean Coder: A Code of Conduct for Professional Programmers,0,A guide to professional conduct in the field of software engineering,FSYfqZIBTvBCTLU88r8R,prentice hall,[ robert c. martin ],20


### bool.must_not
The clause (query) must not appear in the matching documents. Because this query also runs in filter context, no scores are calculated; the filter just determines if a document is included in the results or not.

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Bool(b => b
            .MustNot(m => m
                .Range(r => r.NumberRange(nr => nr.Field(f => f.num_reviews).Lte(45)))
            )
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
Clean Code: A Handbook of Agile Software Craftsmanship,0,"A guide to writing code that is easy to read, understand and maintain",ESYfqZIBTvBCTLU88r8R,prentice hall,[ robert c. martin ],55
JavaScript: The Good Parts,0,A deep dive into the parts of JavaScript that are essential to writing maintainable code,FiYfqZIBTvBCTLU88r8R,oreilly,[ douglas crockford ],51


### Using Filters with Queries
Filters are often added to search queries with the intention of limiting the search to a subset of the documents. A filter can cleanly eliminate documents from a search, without altering the relevance scores of the results.

The next example returns books that have the word "javascript" in their title, only among the books that have more than 45 reviews.

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => s
    .Query(q => q
        .Bool(b => b
            .Must(m => m
                .Match(t => t
                    .Field(f => f.Title).Query("javascript")
                )
            )
            .MustNot(m => m
                .Range(r => r.NumberRange(nr => nr.Field(f => f.num_reviews).Lte(45)))
            )
        )
    )
);

DumpRequest(searchResponse);
PrettyPrint(searchResponse);

Title,Score,Summary,Id,Publisher,Authors,Reviews
JavaScript: The Good Parts,1.7064086,A deep dive into the parts of JavaScript that are essential to writing maintainable code,FiYfqZIBTvBCTLU88r8R,oreilly,[ douglas crockford ],51
